# AI Training
## Edge model training

In [5]:
!python3 -m pip install tensorflow
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Input, Model, layers
from sklearn.model_selection import train_test_split

# Load and preprocess data
RAIN_THRESHHOLD = 0.5
data = pd.read_csv('weather_data.csv', low_memory=False)
data = data.sample(frac=1, random_state=2).reset_index(drop=True)
data = data.replace('', float('nan')).dropna()

# Encode rain to 1 or 0 depending on threshold
y = data['rain'].apply(lambda x: 1 if x > RAIN_THRESHHOLD else 0)
X = data.drop(columns=['rain', 'date'])

# Encode categorical columns
for col in X.select_dtypes(include=['object']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

# Ensure all data is numeric
X = X.astype(float)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape y to (num_samples, 1) for regression
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

# Define the model
inputs = Input(shape=(X_train.shape[1],))
x = layers.Dense(64, activation='relu', name='e1')(inputs)
x_edge = layers.Dense(32, activation='relu', name='e2')(x)
edge_outputs = layers.Dense(1, activation='linear', name='edge_output')(x_edge)
x = layers.Dense(64, activation='relu', name='controlmodel')(x_edge)
x = layers.Dense(128, activation='relu', name='l1')(x)
x = layers.Dense(256, activation='relu', name='l2')(x)
x = layers.Dense(128, activation='relu', name='l3')(x)
x = layers.Dense(64, activation='relu', name='l4')(x)
control_outputs = layers.Dense(1, activation='linear', name='control_output')(x)

# Create and compile the combined model
combined_model = Model(inputs=inputs, outputs=[edge_outputs, control_outputs])
combined_model.compile(optimizer='adam',
                       loss={'edge_output': 'mean_squared_error',
                             'control_output': 'mean_squared_error'},
                       metrics={'edge_output': 'mae',
                                'control_output': 'mae'})

# Train the combined model
combined_model.fit(X_train, [y_train, y_train], 
                   epochs=10, batch_size=32, 
                   validation_data=(X_test, [y_test, y_test]))

# Define the edge_model
edge_model = Model(inputs=inputs, outputs=[combined_model.get_layer('e2').output, edge_outputs])

# x§This needs training 
# Set last as untrainable, then train last ones 
 
# Define the control_model
control_model_input = Input(shape=(32,))  # Shape of the intermediate layer output
z = combined_model.get_layer('controlmodel')(control_model_input)
z = combined_model.get_layer('l1')(z)
z = combined_model.get_layer('l2')(z)
z = combined_model.get_layer('l3')(z)
z = combined_model.get_layer('l4')(z)
control_model_outputs = combined_model.get_layer('control_output')(z)
control_model = Model(inputs=control_model_input, outputs=control_model_outputs)

# Inspect the models
edge_model.summary()
control_model.summary()


Epoch 1/10
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - control_output_loss: 3.0356 - control_output_mae: 0.4235 - edge_output_loss: 512.7975 - edge_output_mae: 5.9870 - loss: 515.8332 - val_control_output_loss: 0.0691 - val_control_output_mae: 0.2191 - val_edge_output_loss: 0.1491 - val_edge_output_mae: 0.2838 - val_loss: 0.2182
Epoch 2/10
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - control_output_loss: 0.0631 - control_output_mae: 0.1714 - edge_output_loss: 0.1556 - edge_output_mae: 0.2937 - loss: 0.2187 - val_control_output_loss: 0.0504 - val_control_output_mae: 0.1263 - val_edge_output_loss: 1.0355 - val_edge_output_mae: 0.8458 - val_loss: 1.0858
Epoch 3/10
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - control_output_loss: 0.0526 - control_output_mae: 0.1370 - edge_output_loss: 0.7320 - edge_output_mae: 0.6409 - loss: 0.7846 - val_control_output_loss: 0.0464 - val_control_output_mae: 0.1076 - val_edge_output_loss: 0.1644 - val_edge_output_mae: 0.3673 - val_loss: 0.2108
Epoch 4/10
3

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 19)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e1 (Dense)                      │ (None, 64)             │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e2 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ edge_output (Dense)             │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,393 (13.25 KB)

 Trainable params: 3,393 (13.25 KB)

...

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ controlmodel (Dense)            │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l1 (Dense)                      │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l2 (Dense)                      │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l3 (Dense)                      │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l4 (Dense)                      │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ control_output (Dense)          │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 84,673 (330.75 KB)

 Trainable params: 84,673 (330.75 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:

results = combined_model.evaluate(X_test, [y_test, y_test])
print("Test Loss:", results[0])
print("Edge Output - MAE:", results[1])
print("Edge Output - MSE - How much its off by on average:", results[2])
print("Control Output - MAE:", results[3])
print("Control Output - MSE:", results[4])

796/796 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step - control_output_loss: 0.0468 - control_output_mae: 0.0913 - edge_output_loss: 0.0480 - edge_output_mae: 0.1352 - loss: 0.0948
Test Loss: 0.09326030313968658
Edge Output - MAE: 0.04743298888206482
Edge Output - MSE - How much its off by on average: 0.04583030194044113
Control Output - MAE: 0.09009821712970734
Control Output - MSE: 0.13423798978328705


In [14]:
edge_model.export("expt")
converter = tf.lite.TFLiteConverter.from_saved_model('./expt')
tflite_model = converter.convert()
with open('../deployment/models/edge_model.tflite', 'wb') as f:
  f.write(tflite_model)
    
control_model.save('../deployment/models/control_model.keras')


INFO:tensorflow:Assets written to: expt/assets


INFO:tensorflow:Assets written to: expt/assets


Saved artifact at 'expt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 19), dtype=tf.float32, name='keras_tensor_17')
Output Type:
  List[TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)]
Captures:
  4466391952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4466392912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13020905936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13020907664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13020907472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13020908432: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1740699871.651628   73505 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1740699871.652764   73505 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-02-27 23:44:31.655078: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./expt
2025-02-27 23:44:31.655532: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-02-27 23:44:31.655539: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: ./expt
2025-02-27 23:44:31.665546: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-02-27 23:44:31.719843: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: ./expt
2025-02-27 23:44:31.724094: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 69017 microseconds.


# On the AI Edge Server

In [202]:
from ai_edge_litert.interpreter import Interpreter
interpreter = Interpreter(model_path='../deployment/models/edge_model.tflite')
signatures = interpreter.get_signature_list()
interpreter.allocate_tensors()

# FEED IN THE INPUT DATA
input_details = interpreter.get_input_details()
input_data = np.array([[0,0,8.7,0,8.6,8.5,11.1,99,1003.1,2,12,2,350,61,66,0.0,15000,45,8]], dtype=np.float32)
# Set input tensor
input_index = input_details[0]['index']
interpreter.set_tensor(input_index, input_data)
# GET THE OUTPUT DATA
interpreter.invoke()
output_details = interpreter.get_output_details()
rain_data = interpreter.get_tensor(output_details[0]['index'])
node_data = interpreter.get_tensor(output_details[1]['index'])
node_data = node_data[0] # Send this to control
prob_rain = rain_data[0]


# Determine if it's raining
RAIN_THRESHOLD = 0.5
prob_rain
node_data

array([  25.881882,    0.      ,  239.48932 ,    0.      ,    0.      ,
         88.81821 ,  366.61047 ,  504.92468 ,    0.      ,  456.74963 ,
       1176.1016  ,    0.      ,  898.9954  , 1364.763   ,  652.17786 ,
          0.      ,  716.0247  ,    0.      ,    0.      ,    0.      ,
          0.      ,    0.      ,    0.      ,    0.      ,    0.      ,
          0.      ,    0.      ,    0.      ,    0.      ,   44.27742 ,
          0.      ,    0.      ], dtype=float32)

# Control Model
## On control model

In [203]:
import numpy as np

## Vars that need to be defined for the function 
# Recieve node_data, data_result
BATCH_SIZE = 1 
X_data = []  
Y_data = []  
data_result = 0.4
data_result = np.array([data_result])

edge_model_result = node_data
edge_model_result = np.reshape(edge_model_result, (-1, 32)) 

control_predictions = control_model.predict(edge_model_result)
print("Control predictions:", control_predictions)

#IF data result exists, add it and intermediate layer to batches
if data_result.size > 0: 
#Add data_result to Y and beginnging nodes to X
    X_data.append(edge_model_result) 
    Y_data.append(data_result) 
    if len(X_data) >= BATCH_SIZE:
        numpy_X_data = np.vstack(X_data)
        numpy_Y_data = np.vstack(Y_data)  

        # Clear lists for the next batch
        X_data.clear()
        Y_data.clear()

        # Compile and train the control_model
        control_model.compile(optimizer='adam',
                              loss='mean_squared_error',
                              metrics=['mae'])
        print("Training Model")
        history = control_model.fit(numpy_X_data, numpy_Y_data,
                                    epochs=1, batch_size=BATCH_SIZE)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Control predictions: [[7.4844575]]
Training Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 580ms/step - loss: 50.1895 - mae: 7.0845


In [7]:
!pip show tensorflow

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, termcolor, typing-extensions, wrapt
Required-by: 
